# BERT

## Requirements

In [ ]:
pip install -r requirements.txt

In [21]:
# Imports
import numpy as np
import pandas as pd
import torch
import transformers as ppb  # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

from statistics import mean

import _helpers as hp

## Load Data
For chosing which ASR data change the variable by the key in the paths dictionary

In [22]:
# Defining paths for our data. "ASR" means the TDNNF-LFMMI method, and "new_ASR" means the Wave2vec method.
paths = {
    "ground_truth": "snips/merged_GT_data.csv",
    "ASR": "snips/ASR_with_labels.csv",
    "new_ASR": "snips/new_ASR_with_labels.csv",
    "new_ASR_Autocorrect": "snips/new_ASR_Autocorrection_with_labels.csv",
}

# Chosing which ASR data we will use
ASR_data = pd.read_csv(paths["new_ASR"])

# Groundtruth data
GT_data = pd.read_csv(paths["ground_truth"])

## Import pre-trained DistilBERT model and tokenizer

DistilBERT is a small, fast, cheap and light Transformer model. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT's performances

In [23]:
model_class, tokenizer_class, pretrained_weights = (
    ppb.DistilBertModel,
    ppb.DistilBertTokenizer,
    "distilbert-base-uncased",
)

## Import pre-trained BERT model and tokenizer

If we want to use BERT instead of DistilBERT we comment the previous model importation and use this one instead:

In [24]:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-small-uncased')

In [25]:
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Tokenization

In [26]:
tokenized_GT_data = GT_data["transcript"].apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True))
)

tokenized_ASR_data = ASR_data["transcript"].apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True))
)

The problem now is that the tokenized vectors ar not with the name size

In [27]:
if max(tokenized_GT_data.apply(len)) > max(tokenized_ASR_data.apply(len)):
    _max = max(tokenized_GT_data.apply(len))
    _min = min(tokenized_GT_data.apply(len))
else:
    _max = max(tokenized_ASR_data.apply(len))
    _min = min(tokenized_ASR_data.apply(len))

print("MX:", _max)
print("MIN:", _min)

MX: 20
MIN: 4


lets fix that with a simple padding function

In [28]:
padded_GT_data = np.array(
    list(tokenized_GT_data.apply(lambda x: hp.padding_func(x, _max)))
)
padded_ASR_data = np.array(
    list(tokenized_ASR_data.apply(lambda x: hp.padding_func(x, _max)))
)
print(padded_GT_data.shape)
print(padded_ASR_data.shape)

(1660, 20)
(1660, 20)


# Masking

If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [29]:
attention_mask_GT = np.where(padded_GT_data != 0, 1, 0)

attention_mask_ASR = np.where(padded_ASR_data != 0, 1, 0)

# Processing with DistrilBert

In [30]:
input_ids_GT = torch.tensor(padded_GT_data)
attention_mask_GT = torch.tensor(attention_mask_GT)

with torch.no_grad():
    last_hidden_states_GT = model(input_ids_GT, attention_mask=attention_mask_GT)

In [31]:
input_ids_ASR = torch.tensor(padded_ASR_data)
attention_mask_ASR = torch.tensor(attention_mask_ASR)

with torch.no_grad():
    last_hidden_states_ASR = model(input_ids_ASR, attention_mask=attention_mask_ASR)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence

basically [CLS] contains all information of the sentence and representing the sentence-level classification.

In [32]:
# Taking the CLS features of each sentence
features_GT = last_hidden_states_GT[0][:, 0, :].numpy()
features_GT.shape

(1660, 768)

In [33]:
# Taking the CLS features of each sentence
features_ASR = last_hidden_states_ASR[0][:, 0, :].numpy()
features_ASR.shape

(1660, 768)

In [34]:
labels_GT = GT_data["user_action_num"]
labels_ASR = ASR_data["user_action_num"]

# Classifiers

## Train/Test Split

The function beneath provides the features and labels needed for testing. Using the loaded ASR or not (then using ground truth data) is decided by input. As standard we use the ASR dataset.

In [35]:
def get_train_test_data(type_of_dataset="ASR", train_size=0.9):
    """Retrieves the relevant dataset and splits according to parameter"""
    # If ASR, give ASR features and labels
    if type_of_dataset == "ASR":
        train_features, test_features, train_labels, test_labels = train_test_split(
            features_ASR, labels_ASR, train_size=train_size
        )
    # If the dataset is not the ASR data, use the ground truth data
    else:
        train_features, test_features, train_labels, test_labels = train_test_split(
            features_GT, labels_GT, train_size=train_size
        )

    return train_features, test_features, train_labels, test_labels


def run_classifier(
    classifier_pipe, type_of_dataset="ASR", train_size=0.9, number_of_times=100
):
    """For running the classifiers multiple times, and returning mean accuracy score. Wraps around get_train_test_data"""
    mean_score_list = []
    n = number_of_times
    for i in range(n):
        train_features, test_features, train_labels, test_labels = get_train_test_data(
            type_of_dataset="ASR"
        )
        classifier_pipe.fit(train_features, train_labels)

        classifier_pred_labels = classifier_pipe.predict(test_features)  # predictions

        classifier_score = classifier_pipe.score(test_features, test_labels)  # accuracy

        mean_score_list.append(classifier_score)
    mean_score = mean(mean_score_list)
    return mean_score, classifier_pred_labels, classifier_score, test_labels

## Logistic Regression

In [36]:
lgr = LogisticRegression(
    C=0.6, max_iter=1000, penalty="l2", solver="liblinear"
)  # Create the classification model

lgr_pipe = make_pipeline(preprocessing.StandardScaler(), lgr)  # Scale feature space

mean_score, lgr_pred_labels, lgr_score, test_labels = run_classifier(
    classifier_pipe=lgr_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.777


### Model Evaluation

In [37]:
print(
    classification_report(
        test_labels,
        lgr_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, lgr_pred_labels))

print("\nACCURACY:", lgr_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.79      0.68      0.73        34
     SwitchLightOn       0.71      0.77      0.74        22
IncreaseBrightness       0.81      0.79      0.80        28
DecreaseBrightness       0.74      0.68      0.71        25
SetLightBrightness       0.84      0.90      0.87        30
     SetLightColor       0.74      0.85      0.79        27

          accuracy                           0.78       166
         macro avg       0.77      0.78      0.77       166
      weighted avg       0.78      0.78      0.78       166

[[23  4  3  3  1  0]
 [ 1 17  1  1  1  1]
 [ 1  0 22  1  0  4]
 [ 1  2  1 17  2  2]
 [ 1  0  0  1 27  1]
 [ 2  1  0  0  1 23]]

ACCURACY: 0.7771084337349398


# Naive Bayes

### Gaussian Naive Bayes

In [38]:
gnb = GaussianNB()  # Create the classification model

gnb_pipe = make_pipeline(preprocessing.StandardScaler(), gnb)  # Scale feature space

mean_score, gnb_pred_labels, gnb_score, test_labels = run_classifier(
    classifier_pipe=gnb_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.608


### Model Evaluation

In [39]:
print(
    classification_report(
        test_labels,
        gnb_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, gnb_pred_labels))

print("\nACCURACY:", gnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.80      0.30      0.43        27
     SwitchLightOn       0.39      0.76      0.51        25
IncreaseBrightness       0.71      0.52      0.60        23
DecreaseBrightness       0.32      0.29      0.30        21
SetLightBrightness       0.79      0.83      0.81        36
     SetLightColor       0.79      0.76      0.78        34

          accuracy                           0.61       166
         macro avg       0.63      0.58      0.57       166
      weighted avg       0.66      0.61      0.60       166

[[ 8 12  0  4  1  2]
 [ 1 19  0  3  2  0]
 [ 0  5 12  3  1  2]
 [ 0  9  3  6  2  1]
 [ 0  1  0  3 30  2]
 [ 1  3  2  0  2 26]]

ACCURACY: 0.608433734939759


## SVM

In [40]:
svm = SVC()  # Create the classification model

svm_pipe = make_pipeline(preprocessing.StandardScaler(), svm)  # Scale feature space

mean_score_list = []

mean_score, svm_pred_labels, svm_score, test_labels = run_classifier(
    classifier_pipe=svm_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.729


### Model Evaluation

In [41]:
print(
    classification_report(
        test_labels,
        svm_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, svm_pred_labels))

print("\nACCURACY:", svm_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.61      0.76      0.68        25
     SwitchLightOn       0.57      0.57      0.57        28
IncreaseBrightness       0.87      0.50      0.63        26
DecreaseBrightness       0.67      0.73      0.70        22
SetLightBrightness       0.82      0.93      0.87        30
     SetLightColor       0.85      0.83      0.84        35

          accuracy                           0.73       166
         macro avg       0.73      0.72      0.72       166
      weighted avg       0.74      0.73      0.73       166

[[19  3  0  1  2  0]
 [ 6 16  0  3  2  1]
 [ 3  5 13  2  1  2]
 [ 2  1  2 16  0  1]
 [ 0  0  0  1 28  1]
 [ 1  3  0  1  1 29]]

ACCURACY: 0.7289156626506024


## NEURAL NETWORK

In [55]:
mlp = MLPClassifier(
    hidden_layer_sizes=(400, 100), activation="relu", solver="adam", max_iter=5000
)  # Create the classification model

mlp_pipe = make_pipeline(preprocessing.StandardScaler(), mlp)  # Scale feature space

mean_score, mlp_pred_labels, mlp_score, test_labels = run_classifier(
    classifier_pipe=mlp_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.751


### Model Evaluation

In [43]:
print(
    classification_report(
        test_labels,
        mlp_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, mlp_pred_labels))

print("\nACCURACY:", mlp_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.68      0.71      0.69        24
     SwitchLightOn       0.48      0.71      0.57        17
IncreaseBrightness       0.70      0.75      0.72        28
DecreaseBrightness       0.56      0.37      0.44        27
SetLightBrightness       0.86      0.86      0.86        36
     SetLightColor       0.78      0.74      0.76        34

          accuracy                           0.70       166
         macro avg       0.68      0.69      0.68       166
      weighted avg       0.70      0.70      0.70       166

[[17  3  2  2  0  0]
 [ 1 12  0  0  2  2]
 [ 1  3 21  2  0  1]
 [ 4  5  5 10  0  3]
 [ 1  1  1  1 31  1]
 [ 1  1  1  3  3 25]]

ACCURACY: 0.6987951807228916


## Try your self

In [44]:
def user_friendly(sentence, mdl):
    """return action from sentence"""
    sent_token = tokenizer.encode(sentence, add_special_tokens=True)
    sent_pad = np.array([sent_token + [0] * (_max - len(sent_token))])
    sent_att_mask = np.where(sent_pad != 0, 1, 0)
    sent_input_ids = torch.tensor(sent_pad)
    sent_attention_mask = torch.tensor(sent_att_mask)

    with torch.no_grad():
        sent_last_hidden_states = model(
            sent_input_ids, attention_mask=sent_attention_mask
        )

    feature = sent_last_hidden_states[0][:, 0, :].numpy()

    prediction = mdl.predict(feature)
    user_action = hp.indx2action(prediction)
    return user_action

In [45]:
user_friendly("red light to the room", lgr_pipe)

['SetLightColor']